# Starbucks Capstone Challenge

### Introduction

This data set contains simulated data that mimics customer behavior on the Starbucks rewards mobile app. Once every few days, Starbucks sends out an offer to users of the mobile app. An offer can be merely an advertisement for a drink or an actual offer such as a discount or BOGO (buy one get one free). Some users might not receive any offer during certain weeks. 

Not all users receive the same offer, and that is the challenge to solve with this data set.

Your task is to combine transaction, demographic and offer data to determine which demographic groups respond best to which offer type. This data set is a simplified version of the real Starbucks app because the underlying simulator only has one product whereas Starbucks actually sells dozens of products.

Every offer has a validity period before the offer expires. As an example, a BOGO offer might be valid for only 5 days. You'll see in the data set that informational offers have a validity period even though these ads are merely providing information about a product; for example, if an informational offer has 7 days of validity, you can assume the customer is feeling the influence of the offer for 7 days after receiving the advertisement.

You'll be given transactional data showing user purchases made on the app including the timestamp of purchase and the amount of money spent on a purchase. This transactional data also has a record for each offer that a user receives as well as a record for when a user actually views the offer. There are also records for when a user completes an offer. 

Keep in mind as well that someone using the app might make a purchase through the app without having received an offer or seen an offer.

### Example

To give an example, a user could receive a discount offer buy 10 dollars get 2 off on Monday. The offer is valid for 10 days from receipt. If the customer accumulates at least 10 dollars in purchases during the validity period, the customer completes the offer.

However, there are a few things to watch out for in this data set. Customers do not opt into the offers that they receive; in other words, a user can receive an offer, never actually view the offer, and still complete the offer. For example, a user might receive the "buy 10 dollars get 2 dollars off offer", but the user never opens the offer during the 10 day validity period. The customer spends 15 dollars during those ten days. There will be an offer completion record in the data set; however, the customer was not influenced by the offer because the customer never viewed the offer.

### Cleaning

This makes data cleaning especially important and tricky.

You'll also want to take into account that some demographic groups will make purchases even if they don't receive an offer. From a business perspective, if a customer is going to make a 10 dollar purchase without an offer anyway, you wouldn't want to send a buy 10 dollars get 2 dollars off offer. You'll want to try to assess what a certain demographic group will buy when not receiving any offers.

### Final Advice

Because this is a capstone project, you are free to analyze the data any way you see fit. For example, you could build a machine learning model that predicts how much someone will spend based on demographics and offer type. Or you could build a model that predicts whether or not someone will respond to an offer. Or, you don't need to build a machine learning model at all. You could develop a set of heuristics that determine what offer you should send to each customer (i.e., 75 percent of women customers who were 35 years old responded to offer A vs 40 percent from the same demographic to offer B, so send offer A).

# Data Sets

The data is contained in three files:

* portfolio.json - containing offer ids and meta data about each offer (duration, type, etc.)
* profile.json - demographic data for each customer
* transcript.json - records for transactions, offers received, offers viewed, and offers completed

Here is the schema and explanation of each variable in the files:

**portfolio.json**
* id (string) - offer id
* offer_type (string) - type of offer ie BOGO, discount, informational
* difficulty (int) - minimum required spend to complete an offer
* reward (int) - reward given for completing an offer
* duration (int) - time for offer to be open, in days
* channels (list of strings)

**profile.json**
* age (int) - age of the customer 
* became_member_on (int) - date when customer created an app account
* gender (str) - gender of the customer (note some entries contain 'O' for other rather than M or F)
* id (str) - customer id
* income (float) - customer's income

**transcript.json**
* event (str) - record description (ie transaction, offer received, offer viewed, etc.)
* person (str) - customer id
* time (int) - time in hours since start of test. The data begins at time t=0
* value - (dict of strings) - either an offer id or transaction amount depending on the record

**Note:** If you are using the workspace, you will need to go to the terminal and run the command `conda update pandas` before reading in the files. This is because the version of pandas in the workspace cannot read in the transcript.json file correctly, but the newest version of pandas can. You can access the termnal from the orange icon in the top left of this notebook.  

You can see how to access the terminal and how the install works using the two images below.  First you need to access the terminal:

<img src="pic1.png"/>

Then you will want to run the above command:

<img src="pic2.png"/>

Finally, when you enter back into the notebook (use the jupyter icon again), you should be able to run the below cell without any errors.

In [650]:
import pandas as pd
import numpy as np
import math
import json
%matplotlib inline

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import classification_report

# read in the json files
portfolio = pd.read_json('data/portfolio.json', orient='records', lines=True)
profile = pd.read_json('data/profile.json', orient='records', lines=True)
transcript = pd.read_json('data/transcript.json', orient='records', lines=True)

### Explore and Clean Data

**portfolio.json**
* reward (int) - reward given for completing an offer
* channels (list of strings)
* difficulty (int) - minimum required spend to complete an offer
* duration (int) - time for offer to be open, in days
* offer_type (string) - type of offer ie BOGO, discount, informational
* id (string) - offer id

In [652]:
portfolio.head()

,reward,channels,difficulty,duration,offer_type,id
0,10,"[email, mobile, social]",10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd
1,10,"[web, email, mobile, social]",10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0
2,0,"[web, email, mobile]",0,4,informational,3f207df678b143eea3cee63160fa8bed
3,5,"[web, email, mobile]",5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9
4,5,"[web, email]",20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7



**profile.json**
* gender (str) - gender of the customer (note some entries contain 'O' for other rather than M or F)
* age (int) - age of the customer
* id (str) - customer id
* became_member_on (int) - date when customer created an app account
* income (float) - customer's income

In [654]:
profile.head()

,gender,age,id,became_member_on,income
0,None,118,68be06ca386d4c31939f3a4f0e3dd783,20170212,NaN
1,F,55,0610b486422d4921ae7d2bf64640c50b,20170715,112000.0
2,None,118,38fe809add3b4fcf9315a9694bb96ff5,20180712,NaN
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,100000.0
4,None,118,a03223e636434f42ac4c3df47e8bac43,20170804,NaN


**transcript.json**
* person (str) - customer id
* event (str) - record description (ie transaction, offer received, offer viewed, etc.)
* value - (dict of strings) - either an offer id or transaction amount depending on the record
* time (int) - time in hours since start of test. The data begins at time t=0

In [656]:
transcript.head()

,person,event,value,time
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0
1,a03223e636434f42ac4c3df47e8bac43,offer received,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0
2,e2127556f4f64592b11af22de27a7932,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},0
4,68617ca6246f4fbc85e91a2a49552598,offer received,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0


In [657]:
# We have multiple IDs, im going to rename them so its easier to see which relates to offer and which to customer for future joins.
portfolio.rename(columns={'id': 'offer_id'}, inplace=True)
profile.rename(columns={'id': 'customer_id'}, inplace=True)
transcript.rename(columns={'person': 'customer_id'}, inplace=True)

In [658]:
# View them again
portfolio.head()

,reward,channels,difficulty,duration,offer_type,offer_id
0,10,"[email, mobile, social]",10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd
1,10,"[web, email, mobile, social]",10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0
2,0,"[web, email, mobile]",0,4,informational,3f207df678b143eea3cee63160fa8bed
3,5,"[web, email, mobile]",5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9
4,5,"[web, email]",20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7


In [659]:
profile.head()

,gender,age,customer_id,became_member_on,income
0,None,118,68be06ca386d4c31939f3a4f0e3dd783,20170212,NaN
1,F,55,0610b486422d4921ae7d2bf64640c50b,20170715,112000.0
2,None,118,38fe809add3b4fcf9315a9694bb96ff5,20180712,NaN
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,100000.0
4,None,118,a03223e636434f42ac4c3df47e8bac43,20170804,NaN


In [660]:
transcript.head()

,customer_id,event,value,time
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0
1,a03223e636434f42ac4c3df47e8bac43,offer received,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0
2,e2127556f4f64592b11af22de27a7932,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},0
4,68617ca6246f4fbc85e91a2a49552598,offer received,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0


In [661]:
transcript.tail()

,customer_id,event,value,time
306529,b3a1272bc9904337b331bf348c3e8c17,transaction,{'amount': 1.59},714
306530,68213b08d99a4ae1b0dcb72aebd9aa35,transaction,{'amount': 9.53},714
306531,a00058cf10334a308c68e7631c529907,transaction,{'amount': 3.61},714
306532,76ddbd6576844afe811f1a3c0fbb5bec,transaction,{'amount': 3.53},714
306533,c02b10e8752c4d8e9b73f918558531f7,transaction,{'amount': 4.05},714


In [662]:
# Grouping and viewing unique event values
event_counts = transcript['event'].value_counts()
print(event_counts)

event
transaction        138953
offer received      76277
offer viewed        57725
offer completed     33579
Name: count, dtype: int64


In [663]:
# Cleaning the value column - seperating it so we know the offer id to join if needed
# Extracting offer_id from both keys into a single column
transcript['offer_id'] = transcript['value'].apply(
    lambda x: x.get('offer id') if isinstance(x, dict) and 'offer id' in x else (
        x.get('offer_id') if isinstance(x, dict) and 'offer_id' in x else None
    )
)

# Create column with amount
transcript['amount'] = transcript['value'].apply(lambda x: x.get('amount') if isinstance(x, dict) else 0)

In [664]:
# Checking for nulls in my new fields, offer_id and amount
transcript.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 306534 entries, 0 to 306533
Data columns (total 6 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   customer_id  306534 non-null  object 
 1   event        306534 non-null  object 
 2   value        306534 non-null  object 
 3   time         306534 non-null  int64  
 4   offer_id     167581 non-null  object 
 5   amount       138953 non-null  float64
dtypes: float64(1), int64(1), object(4)
memory usage: 14.0+ MB


In [665]:
# Viewing rows where both offer_id and amount are null - should be none
null_rows = transcript[(transcript['offer_id'].isnull()) & (transcript['amount'].isnull())]
print(null_rows)


Empty DataFrame
Columns: [customer_id, event, value, time, offer_id, amount]
Index: []


## Creating a variable to show if a customer has both seen and completed an offer to use in my model.

In [667]:
# Identifying Response (1 = responded, 0 = not responded)
transcript['viewed_flag'] = (transcript['event'] == 'offer viewed').astype(int)
transcript['completed_flag'] = (transcript['event'] == 'offer completed').astype(int)

In [668]:
# Grouping and viewing unique event values to QA
print(transcript['viewed_flag'].value_counts())
print(transcript['completed_flag'].value_counts())

viewed_flag
0    248809
1     57725
Name: count, dtype: int64
completed_flag
0    272955
1     33579
Name: count, dtype: int64


In [669]:
transcript.head()

,customer_id,event,value,time,offer_id,amount,viewed_flag,completed_flag
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0,9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN,0,0
1,a03223e636434f42ac4c3df47e8bac43,offer received,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0,0b1e1539f2cc45b7b9fa7c272da2e1d7,NaN,0,0
2,e2127556f4f64592b11af22de27a7932,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0,2906b810c7d4411798c6938adc9daaa5,NaN,0,0
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},0,fafdcd668e3743c1bb461111dcafc2a4,NaN,0,0
4,68617ca6246f4fbc85e91a2a49552598,offer received,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0,4d5c57ea9a6940dd891ad53e9dbe8da0,NaN,0,0


In [670]:
#Taking viewed records
viewed=transcript[transcript['viewed_flag']>0]
#print(viewed)

#Taking completed records
completed=transcript[transcript['completed_flag']>0]
#print(completed)

# Merge together on customer_id and offer_id to get a list of offer that were viewed and completed
completed = completed.merge(viewed, on=['customer_id', 'offer_id'], how='inner')

# Creating deduplicated list of completed offers by customers
completed = completed[['customer_id', 'offer_id']].drop_duplicates()

# Adding completed_flag column with value 1
completed['completed_flag_final'] = 1

In [671]:
completed.head()

,customer_id,offer_id,completed_flag_final
0,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,2906b810c7d4411798c6938adc9daaa5,1
1,fe97aa22dd3e48c8b143116a8403dd52,fafdcd668e3743c1bb461111dcafc2a4,1
2,629fc02d56414d91bca360decdfa9288,9b98b8c7a33c4b65b9aebfe6a799e6d9,1
3,676506bad68e4161b9bbaffeb039626b,ae264e3637204a6fb9bb56bc8210ddfd,1
4,8f7dd3b2afe14c078eb4f6e6fe4ba97d,4d5c57ea9a6940dd891ad53e9dbe8da0,1


In [672]:
# Drop completed and viewed columns from transcript data
transcript.drop(columns=['completed_flag', 'viewed_flag'], inplace=True)

# Merge this on to the original data
transcript = transcript.merge(completed, on=['customer_id', 'offer_id'], how='left')

#Fill NAs
# Recommended way to fill NaNs without chained assignment warning
transcript['completed_flag_final'] = transcript['completed_flag_final'].fillna(0)

In [673]:
transcript.completed_flag_final.value_counts()

completed_flag_final
0.0    217569
1.0     88965
Name: count, dtype: int64

## Looking at deduping the data for ease

In [695]:
transcript_dedup = transcript

# Drop columns I dont need
transcript_dedup.drop(columns=['event', 'value', 'amount','time'], inplace=True)

#deduplicate based on all
transcript_dedup = transcript_dedup.drop_duplicates()

In [697]:
# Group by 'offer_id' and get max of 'flag' and 'customer_id'
transcript_dedup = transcript_dedup.groupby('offer_id').agg({
    'completed_flag_final': 'max',
    'customer_id': 'max'
}).reset_index()


In [699]:
# Checking
duplicate_offer_ids = transcript_dedup['offer_id'].duplicated().sum()
print(f"Number of duplicate offer_id values: {duplicate_offer_ids}")


Number of duplicate offer_id values: 0


In [701]:
transcript_dedup.head()

,offer_id,completed_flag_final,customer_id
0,0b1e1539f2cc45b7b9fa7c272da2e1d7,1.0,ffff82501cea40309d5fdd7edcca4a07
1,2298d6c36e964ae4a3e7e9706d1fb8c2,1.0,ffd53e5ba61d41378b66efc00d9f9682
2,2906b810c7d4411798c6938adc9daaa5,1.0,ffff82501cea40309d5fdd7edcca4a07
3,3f207df678b143eea3cee63160fa8bed,0.0,fff8957ea8b240a6b5e634b6ee8eafcf
4,4d5c57ea9a6940dd891ad53e9dbe8da0,1.0,fff8957ea8b240a6b5e634b6ee8eafcf


## Merging with the other datasets

In [703]:
# Merging Data to allow for more feature choice
# Keep in mind this data is sitting at an offer level - de-dup later to make more sence to me
df = transcript_dedup.merge(profile, on='customer_id', how='left')

df['offer_id'] = df['offer_id'].astype(str).str.strip()
portfolio['offer_id'] = portfolio['offer_id'].astype(str).str.strip()

df = df.merge(portfolio, on='offer_id', how='left')

In [705]:
df.head()

,offer_id,completed_flag_final,customer_id,gender,age,became_member_on,income,reward,channels,difficulty,duration,offer_type
0,0b1e1539f2cc45b7b9fa7c272da2e1d7,1.0,ffff82501cea40309d5fdd7edcca4a07,F,45,20161125,62000.0,5,"[web, email]",20,10,discount
1,2298d6c36e964ae4a3e7e9706d1fb8c2,1.0,ffd53e5ba61d41378b66efc00d9f9682,M,40,20170216,32000.0,3,"[web, email, mobile, social]",7,7,discount
2,2906b810c7d4411798c6938adc9daaa5,1.0,ffff82501cea40309d5fdd7edcca4a07,F,45,20161125,62000.0,2,"[web, email, mobile]",10,7,discount
3,3f207df678b143eea3cee63160fa8bed,0.0,fff8957ea8b240a6b5e634b6ee8eafcf,M,71,20180218,56000.0,0,"[web, email, mobile]",0,4,informational
4,4d5c57ea9a6940dd891ad53e9dbe8da0,1.0,fff8957ea8b240a6b5e634b6ee8eafcf,M,71,20180218,56000.0,10,"[web, email, mobile, social]",10,5,bogo


In [707]:
# QA - Check for dups again after merging
duplicate_offer_ids = df['offer_id'].duplicated().sum()
print(f"Number of duplicate offer_id values: {duplicate_offer_ids}")

Number of duplicate offer_id values: 0


## Feature Engineering

In [709]:
# Adding a feature for how many days the customer has been a member. 
df['member_days'] = (pd.to_datetime('2022-01-01') - pd.to_datetime(df['became_member_on'], format='%Y%m%d')).dt.days

In [727]:
# Adding a feature to band income levels in to low, medium and high.
df['income_band'] = pd.qcut(df['income'], q=3, labels=['Low', 'Medium', 'High'])

In [731]:
# Adding a feature to band ages
# Split into 4 quartiles
df['age_band'] = pd.qcut(df['age'], q=4, labels=['Young', 'Mid-Young', 'Mid-Old', 'Old'])

In [733]:
df.head()

,offer_id,completed_flag_final,customer_id,gender,age,became_member_on,income,reward,channels,difficulty,duration,offer_type,member_days,income_band,age_band
0,0b1e1539f2cc45b7b9fa7c272da2e1d7,1.0,ffff82501cea40309d5fdd7edcca4a07,F,45,20161125,62000.0,5,"[web, email]",20,10,discount,1863,Medium,Mid-Young
1,2298d6c36e964ae4a3e7e9706d1fb8c2,1.0,ffd53e5ba61d41378b66efc00d9f9682,M,40,20170216,32000.0,3,"[web, email, mobile, social]",7,7,discount,1780,Low,Young
2,2906b810c7d4411798c6938adc9daaa5,1.0,ffff82501cea40309d5fdd7edcca4a07,F,45,20161125,62000.0,2,"[web, email, mobile]",10,7,discount,1863,Medium,Mid-Young
3,3f207df678b143eea3cee63160fa8bed,0.0,fff8957ea8b240a6b5e634b6ee8eafcf,M,71,20180218,56000.0,0,"[web, email, mobile]",0,4,informational,1413,Medium,Old
4,4d5c57ea9a6940dd891ad53e9dbe8da0,1.0,fff8957ea8b240a6b5e634b6ee8eafcf,M,71,20180218,56000.0,10,"[web, email, mobile, social]",10,5,bogo,1413,Medium,Old


### Model
Building a model that predicts whether or not someone will respond to an offer using logistic regression.

# Steps
* Split data in to train, test
* Train the model and tune it using cross-validation.
* Evaluate using metrics like accuracy, precision, recall, and F1-score.

In [ ]:

# Selecting Features
X = df[['age', 'income', 'member_days', 'difficulty', 'reward', 'duration']]
y = df['response']

In [ ]:

# Define features and target
X = df[['offer_id', 'customer_id', 'feature1', 'feature2']]
y = df['flag']

# Preprocessing for categorical features
categorical_features = ['offer_id', 'customer_id', 'feature2']
numerical_features = ['feature1']

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ],
    remainder='passthrough'  # keep numerical features
)

# Define model pipeline
model = make_pipeline(preprocessor, LogisticRegression())

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Fit model
model.fit(X_train, y_train)




In [ ]:
# Predict and evaluate
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))